#Addressing Data Anomalies

In [196]:
from google.colab import auth
auth.authenticate_user()

In [197]:
#Import everything we need
import pandas as pd
import numpy as np
import pandas_gbq
from google.cloud import bigquery

In [198]:
#Initialize variables
project_id = "group-5-448704"
raw_dataset = "football_dataset_raw"
stg_dataset = "football_dataset_stg"
int_dataset = "football_dataset_int"
client = bigquery.Client(project=project_id)

In [199]:
dataset_id = int_dataset
dataset = bigquery.Dataset(f"{client.project}.{dataset_id}")
dataset.location = "us-central1"
client.create_dataset(dataset, exists_ok=True)

Dataset(DatasetReference('group-5-448704', 'football_dataset_int'))

##Anomaly 9: Stadium names were being indentified across different tables with redundant indentifiers in table stadiums an table spreadspoke_scores.

In [200]:
# Load tables into DataFrames
stadiums = pd.read_gbq(f"SELECT * FROM {raw_dataset}.stadiums", project_id=project_id)
spreadspoke_scores = pd.read_gbq(f"SELECT * FROM {raw_dataset}.spreadspoke_scores", project_id=project_id)

# Create a mapping table
mapping = pd.DataFrame({
    "original_name": pd.concat([
        stadiums["stadium_name"],
        spreadspoke_scores["stadium"]
    ]).unique()
})
mapping["universal_stadium_id"] = range(1, len(mapping) + 1)

# Upload mapping table to BigQuery
mapping.to_gbq(f"{int_dataset}.stadium_mapping_temp_", project_id=project_id, if_exists="replace")

# Update stadiums table
stadiums_info_updated = pd.merge(
    stadiums,
    mapping[["original_name", "universal_stadium_id"]],
    left_on="stadium_name",
    right_on="original_name"
)
stadiums_info_updated = stadiums.drop(columns=["stadium_name"])
stadiums_info_updated = stadiums.rename(columns={"original_name":"stadium_name"})
stadiums_info_updated.to_gbq(f"{int_dataset}.Stadiums_Updated", project_id=project_id, if_exists="replace")

# Update stadiumDetails table
spreadspoke_scores_updated = pd.merge(
    spreadspoke_scores,
    mapping[["original_name", "universal_stadium_id"]],
    left_on="stadium",
    right_on="original_name"
)
spreadspoke_scores_updated = spreadspoke_scores_updated.drop(columns=["stadium"])
spreadspoke_scores_updated = spreadspoke_scores_updated.rename(columns={"original_name":"stadium_name"})
spreadspoke_scores_updated.to_gbq(f"{int_dataset}.Spreadspoke_Scores_Updated", project_id=project_id, if_exists="replace")

<ipython-input-200-1c15da8ddadd>:2: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  stadiums = pd.read_gbq(f"SELECT * FROM {raw_dataset}.stadiums", project_id=project_id)
<ipython-input-200-1c15da8ddadd>:3: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  spreadspoke_scores = pd.read_gbq(f"SELECT * FROM {raw_dataset}.spreadspoke_scores", project_id=project_id)
<ipython-input-200-1c15da8ddadd>:15: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  mapping.to_gbq(f"{int_dataset}.stadium_mapping_temp_", project_id=project_id, if_exists="replace")
100%|██████████| 1/1 [

##Anomaly 8: Weather types for stadiums in football_dataset_raw had multiple weather types in certain "warm, hot" weathers.

In [201]:
#Load tables into DataFrames
stadiums_df = pd.read_gbq(f"SELECT * FROM {raw_dataset}.stadiums", project_id=project_id)#

#Flatten Data
stadiums_df = (
    stadiums_df.assign(stadium_weather_type=stadiums_df['stadium_weather_type'].str.split(', '))
    .explode('stadium_weather_type')
    .reset_index(drop=True)
)

#Upload
stadiums_df.to_gbq(f"{int_dataset}.Stadiums", project_id=project_id, if_exists="replace")

<ipython-input-201-28a9fc101d3c>:2: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  stadiums_df = pd.read_gbq(f"SELECT * FROM {raw_dataset}.stadiums", project_id=project_id)#
<ipython-input-201-28a9fc101d3c>:12: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  stadiums_df.to_gbq(f"{int_dataset}.Stadiums", project_id=project_id, if_exists="replace")
100%|██████████| 1/1 [00:00<00:00, 5675.65it/s]


##Anomaly 10: Team preferred identification and team conference logical entities were stored in the same table.

In [202]:
#Create Queries
queries = {
    "team_id": """
    CREATE OR REPLACE TABLE football_dataset_int.Team_Identification AS
    SELECT DISTINCT team_name ,team_name_short, team_id, team_id_pfr, _data_source, _load_time
    FROM football_dataset_raw.teams;
    """,

    "team_conference": """
    CREATE OR REPLACE TABLE football_dataset_int.Team_Conference AS
    SELECT DISTINCT team_name, team_conference, team_conference_pre_2002, team_division_pre2002, _data_source, _load_time
    FROM football_dataset_raw.teams;
    """
}

# Execute each query
for table_name, query in queries.items():
    job = client.query(query)
    job.result()



# Filling Empty cells within Stadiums Table

In [203]:
import itertools, json, pandas, pandas_gbq
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel, Part

#Generate AI prompt
prompt = """Here is a list of names.
I want you to check of the name corresponds to an American Football Stadium.
if it does, return the name of the stadium, the state and city it is from, its full address, the year it opened,,
whether it has an open, closed, or retractable roof, its capacity, its longitude, and its latitude.
Return the reuslts as a properly formatted JSON object with only one JSON object per line.
Return only one answer per stadium.
Do include stadiums that have already been closed.
Do not include commas in the capacity field.
Return the opening_date field as an integer.
Return Longitude and Latitude as a signed floating point number.
the roof_type field should be entirely lowercase.
Don't return the records which are not American football stadiums.
Don't return any empty JSON objects.
Don't return an explanation for your answer.

Here are some sample runs:

I give you:
"Fenway Park, Boston, MA"
"SoFi Stadium, Inglewood, CA"
"Allegiant Stadium, Paradise, NV"

You return:
{"name": "SoFi Stadium", "location": "Inglewood, CA", "address": "1001 S. Stadium Dr. Inglewood, CA 90301", "opening_date": "September 8, 2020", "roof_type": "Open", "capacity": 70000, "longitude": "118.3390 W", "Latitude": "33.9535 N"}
{"name": "Allegiant Stadium", "location": "Las Vegas, NV","address": "3333 Al Davis Way Las Vegas, NV 89118", "opening_date": "July 31, 2020","roof_type": "Closed", "capacity": 65000, "longitude": "115.1833 W", "latitude": "36.0909 N"}
"""
#Create model instance and bq client
sql = """select distinct stadium_name, stadium_location from football_dataset_int.Stadiums"""
region = "us-central1"
model_name = "gemini-2.0-flash-001"
bq_client = bigquery.Client()
rows = bq_client.query_and_wait(sql)
vertexai.init(project = project_id, location = region)
model = GenerativeModel(model_name)

#iterate over each row of stadium names, condense into strings of stadium names for batches
batch_size = 30
record_counter = 0
stadium_str = ""
stadiums = []
for row in rows:
  record_counter += 1
  if record_counter == 1:
    stadium_str = f"{row['stadium_name']}, {row['stadium_location']}"
  else:
    stadium_str += f"\n {row['stadium_name']}, {row['stadium_location']}"
  if record_counter == batch_size:
    stadiums.append(stadium_str)
    stadium_str = ""
    record_counter = 0
print(f"{len(stadiums)} batches will be sent to LLM")

table_id = "football_dataset_int.tmp_Stadiums"

#Send batches to LLM with prompt to get response
for i, records in enumerate(stadiums):
  first_stadium = records.split(",")[0]
  print(f"{i}: batch starting with stadium {first_stadium}")

  resp = model.generate_content([records, prompt])
  resp_text = resp.text.replace("```json", "").replace("```", "")
  json_text = resp_text.split("\n")
  json_objs = []
  #Convert response string to JSON object
  for json_str in json_text:
    if json_str in (None, ""):
      continue
    else:
      json_str_clean = json_str.replace("},", "}")
      try:
        json_objs.append(json.loads(json_str))
      except Exception as e:
        print(f"Error converting {json_str} to json:", e)
  print("json_objs:", json_objs)

#Convert JSON to Pandas Dataframe
  try:
    df_raw = pandas.DataFrame(json_objs)
  except Exception as e:
    print("Error while creating df_raw", e)
    break
#Drop duplicate dataframe rows
  try:
    print(df_raw.columns)
    df_unique = df_raw.drop_duplicates(subset = ["name", "address"], keep = "last")
  except Exception as e:
    print("Error while creating df_unique:", e)
    break
#Write dataframe to bigquery, appending if table already created
  try:
    if i == 0:
      pandas_gbq.to_gbq(df_unique, table_id, project_id = project_id, if_exists = "replace")
    else:
      pandas_gbq.to_gbq(df_unique, table_id, project_id = project_id, if_exists = "append")
  except Exception as e:
    print("Error while writing to BQ:", e, "\n Error caused by: ", df_unique)





4 batches will be sent to LLM
0: batch starting with stadium Fenway Park
json_objs: [{'name': 'Jack Murphy Stadium', 'location': 'San Diego, CA', 'address': '9449 Friars Rd, San Diego, CA 92108', 'opening_date': 1967, 'roof_type': 'open', 'capacity': 70561, 'longitude': -117.1185, 'latitude': 32.7832}, {'name': 'Legion Field', 'location': 'Birmingham, AL', 'address': '400 Graymont Ave W, Birmingham, AL 35204', 'opening_date': 1927, 'roof_type': 'open', 'capacity': 71594, 'longitude': -86.8222, 'latitude': 33.5156}, {'name': 'Stanford Stadium', 'location': 'Palo Alto, CA', 'address': '625 Nelson Rd, Stanford, CA 94305', 'opening_date': 1921, 'roof_type': 'open', 'capacity': 50424, 'longitude': -122.1697, 'latitude': 37.4345}, {'name': 'Tampa Stadium', 'location': 'Tampa, FL', 'address': '4201 N Dale Mabry Hwy, Tampa, FL 33607', 'opening_date': 1967, 'roof_type': 'open', 'capacity': 74301, 'longitude': -82.5017, 'latitude': 27.9759}, {'name': 'Rose Bowl', 'location': 'Pasadena, CA', 'add

100%|██████████| 1/1 [00:00<00:00, 5817.34it/s]


1: batch starting with stadium Ralph Wilson Stadium
json_objs: [{'name': 'Ralph Wilson Stadium', 'location': 'Orchard Park, NY', 'address': '1 Bills Dr, Orchard Park, NY 14127', 'opening_date': 1973, 'roof_type': 'open', 'capacity': 71608, 'longitude': -78.7869, 'latitude': 42.7739}, {'name': 'New Era Field', 'location': 'Orchard Park, NY', 'address': '1 Bills Dr, Orchard Park, NY 14127', 'opening_date': 1973, 'roof_type': 'open', 'capacity': 71608, 'longitude': -78.7869, 'latitude': 42.7739}, {'name': 'War Memorial Stadium', 'location': 'Buffalo, NY', 'address': 'Best St & Dodge St, Buffalo, NY 14213', 'opening_date': 1937, 'roof_type': 'open', 'capacity': 42000, 'longitude': -78.8644, 'latitude': 42.8967}, {'name': 'Acrisure Stadium', 'location': 'Pittsburgh, PA', 'address': '100 Art Rooney Ave, Pittsburgh, PA 15212', 'opening_date': 2001, 'roof_type': 'open', 'capacity': 68400, 'longitude': -80.0158, 'latitude': 40.4467}, {'name': 'Heinz Field', 'location': 'Pittsburgh, PA', 'addres

100%|██████████| 1/1 [00:00<00:00, 5957.82it/s]


2: batch starting with stadium FirstEnergy Stadium
json_objs: [{'name': 'FirstEnergy Stadium', 'location': 'Cleveland, OH', 'address': '100 Alfred Lerner Way, Cleveland, OH 44114', 'opening_date': 1999, 'roof_type': 'open', 'capacity': 67895, 'longitude': -81.6967, 'latitude': 41.5062}, {'name': 'Cinergy Field', 'location': 'Cincinnati, OH', 'address': '100 Main St, Cincinnati, OH 45202', 'opening_date': 1970, 'roof_type': 'open', 'capacity': 52952, 'longitude': -84.5067, 'latitude': 39.0978}, {'name': 'Paycor Stadium', 'location': 'Cincinnati, OH', 'address': '1 Paul Brown Stadium, Cincinnati, OH 45202', 'opening_date': 2000, 'roof_type': 'open', 'capacity': 65515, 'longitude': -84.5161, 'latitude': 39.0954}, {'name': 'Paul Brown Stadium', 'location': 'Cincinnati, OH', 'address': '1 Paul Brown Stadium, Cincinnati, OH 45202', 'opening_date': 2000, 'roof_type': 'open', 'capacity': 65515, 'longitude': -84.5161, 'latitude': 39.0954}, {'name': 'Nippert Stadium', 'location': 'Cincinnati, OH

100%|██████████| 1/1 [00:00<00:00, 4466.78it/s]


3: batch starting with stadium Cotton Bowl
json_objs: [{'name': 'AT&T Stadium', 'location': 'Arlington, TX', 'address': '1 AT&T Way, Arlington, TX 76011', 'opening_date': 2009, 'roof_type': 'retractable', 'capacity': 80000, 'longitude': -97.0928, 'latitude': 32.7478}, {'name': 'Houston Astrodome', 'location': 'Houston, TX', 'address': '8400 Kirby Dr, Houston, TX 77054', 'opening_date': 1965, 'roof_type': 'closed', 'capacity': 62439, 'longitude': -95.4108, 'latitude': 29.6847}, {'name': 'NRG Stadium', 'location': 'Houston, TX', 'address': '8825 Kirby Dr, Houston, TX 77054', 'opening_date': 2002, 'roof_type': 'retractable', 'capacity': 72220, 'longitude': -95.4108, 'latitude': 29.6847}, {'name': 'Rice Stadium', 'location': 'Houston, TX', 'address': '6100 Main St, Houston, TX 77005', 'opening_date': 1950, 'roof_type': 'open', 'capacity': 47000, 'longitude': -95.4079, 'latitude': 29.7162}, {'name': 'Alamo Dome', 'location': 'San Antonio, TX', 'address': '100 Montana St, San Antonio, TX 782

100%|██████████| 1/1 [00:00<00:00, 5562.74it/s]


# Check for duplicate entries, then set stadium info to info gathered by LLM

In [204]:
%%bigquery
  select name, address, count(*) as duplicates
  from football_dataset_int.tmp_Stadiums
  group by name, address
  having count(*) > 1

Query is running:   0%|          |

Downloading: |          |

,name,address,duplicates


In [205]:
%%bigquery
  update football_dataset_int.Stadiums s
  set s.stadium_name = t.name, s.stadium_address = t.address, s.stadium_open = t.opening_date, s.stadium_type = t.roof_type, s.stadium_capacity = t.capacity, s.stadium_location = t.location, s.stadium_latitude = t.latitude, s.stadium_longitude = t.longitude
  from football_dataset_int.tmp_Stadiums t
  where s.stadium_name = t.name

Query is running:   0%|          |

""


# Delete Null PK values from stadiums, then remove duplicate PK entries

In [206]:
%%bigquery
  delete from football_dataset_int.Stadiums
  where stadium_name is null or stadium_location is null

Query is running:   0%|          |

""


In [207]:
%%bigquery
  create or replace table football_dataset_int.Stadiums as
     select distinct stadium_name, stadium_location, stadium_open, stadium_close, stadium_type, stadium_address, stadium_weather_station_zipcode, stadium_capacity, stadium_surface, stadium_weather_station, stadium_weather_station_name, stadium_latitude, stadium_longitude, stadium_azimuthangle, stadium_elevation
     from football_dataset_int.Stadiums

Query is running:   0%|          |

""


#Enrich Stadium Table with Reviews Field and Superbowl Count Field in same way as earlier use of LLM.

In [208]:
#Create prompt and sql input for LLM
prompt = """Here is a list of names of American football stadiums.
  I want you to take the name of the stadium and return each stadium's name, its average review score on Google, as well as how many Superbowls that stadium has hosted.
  Return the results as a properly formatted JSON object with only one JSON object per line.
  Return only one answer per stadium.
  Format the results as a JSON object with the schema:
  {"name": string, "review_score": float, "superbowl_count": integer}
  return the name as a string.
  return the review score as a float.
  return the superbowl count as an integer.
  return each JSON object in a separate line.
  do not include an explanation with your answer.
  """
llm_input = "select distinct stadium_name from football_dataset_int.Stadiums"
#Create instance of LLM
rows = bq_client.query_and_wait(llm_input)
vertexai.init(project = project_id, location = region)
model = GenerativeModel(model_name)
enrichment_str = ""
record_counter = 0
#Iterate for row of stadium names, adding all to one string
for row in rows:
  record_counter += 1
  if record_counter == 1:
    enrichment_str = f"{row['stadium_name']}"
  else:
    enrichment_str += f"\n {row['stadium_name']}"
#Get LLM response
resp = model.generate_content([enrichment_str, prompt])
resp_text = resp.text.replace("```json", "").replace("```", "")
json_text = resp_text.split("\n")
json_objs = []
#Convert text to JSON object
for json_str in json_text:
  if json_str in (None, ""):
    continue
  else:
    json_str_clean = json_str.replace("},", "}")
    try:
      json_objs.append(json.loads(json_str_clean))
    except Exception as e:
      print(f"Error converting {json_str_clean} to json:", e)
print("json_objs:", json_objs)
#Convert JSON to Pandas Dataframe
df_raw = pandas.DataFrame.from_records(json_objs)
print(df_raw.columns)
#Write new info to temp table in BQ
table_id = "football_dataset_int.Stadiums_enrichment"
pandas_gbq.to_gbq(df_raw, table_id, project_id = project_id, if_exists = "replace")


json_objs: [{'name': 'Seattle Kingdome', 'review_score': None, 'superbowl_count': 0}, {'name': 'Fenway Park', 'review_score': 4.7, 'superbowl_count': 0}, {'name': 'Allianz Arena', 'review_score': 4.7, 'superbowl_count': 0}, {'name': 'Balboa Stadium', 'review_score': 4.5, 'superbowl_count': 0}, {'name': 'Kezar Stadium', 'review_score': 4.6, 'superbowl_count': 0}, {'name': 'Cotton Bowl', 'review_score': 4.5, 'superbowl_count': 1}, {'name': 'Wrigley Field', 'review_score': 4.7, 'superbowl_count': 0}, {'name': 'Frankfurt Stadium', 'review_score': 4.5, 'superbowl_count': 0}, {'name': 'Stanford Stadium', 'review_score': 4.5, 'superbowl_count': 0}, {'name': 'SoFi Stadium', 'review_score': 4.7, 'superbowl_count': 1}, {'name': 'Rose Bowl', 'review_score': 4.7, 'superbowl_count': 5}, {'name': 'Anaheim Stadium', 'review_score': 4.4, 'superbowl_count': 0}, {'name': 'Jack Murphy Stadium', 'review_score': 4.5, 'superbowl_count': 3}, {'name': 'Allegiant Stadium', 'review_score': 4.7, 'superbowl_count

100%|██████████| 1/1 [00:00<00:00, 5053.38it/s]


# Create new table consisting of everything from previous stadiums table joined by enriched data from LLM

In [209]:
%%bigquery
create or replace table football_dataset_int.Stadiums as
  select stadium_name, stadium_location, stadium_open, stadium_close, stadium_type, stadium_address, stadium_weather_station_zipcode, stadium_capacity, stadium_surface, stadium_weather_station, stadium_weather_station_name, stadium_latitude, stadium_longitude, stadium_azimuthangle, stadium_elevation, review_score, superbowl_count
  from football_dataset_int.Stadiums s
  left join football_dataset_int.Stadiums_enrichment e
  on s.stadium_name = e.name

Query is running:   0%|          |

""


# Adding Data source and load time fields back to data after they were removed earlier. Foreign key establishments are all in separate cells because bigquery was not liking when I tried to group them together in one cell.

In [210]:
%%bigquery
  alter table football_dataset_int.Stadiums add column _data_source array<string>;
  alter table football_dataset_int.Stadiums alter column _data_source set default ["Kaggle", "Gemini"];
  update football_dataset_int.Stadiums set _data_source = ["Kaggle", "Gemini"] where true;

  alter table football_dataset_int.Stadiums add column _load_time timestamp;
  alter table football_dataset_int.Stadiums alter column _load_time set default current_timestamp();
  update football_dataset_int.Stadiums set _load_time = current_timestamp() where true;

Query is running:   0%|          |

""


In [211]:
%%bigquery
  alter table football_dataset_int.Team_Conference add primary key (team_name) not enforced;
  alter table football_dataset_int.Spreadspoke_Scores_Updated add primary key (schedule_date) not enforced;
  alter table football_dataset_int.Stadiums add primary key (stadium_name) not enforced;
  alter table football_dataset_int.Team_Identification add primary key (team_name) not enforced;
  alter table football_dataset_int.Spreadspoke_Scores_Updated add constraint stadium_fk foreign key (stadium_name) references football_dataset_int.Stadiums (stadium_name) not enforced;

Query is running:   0%|          |

""


In [212]:
%%bigquery
alter table football_dataset_int.Spreadspoke_Scores_Updated add constraint team_home_fk foreign key (team_home) references football_dataset_int.Team_Identification (team_name) not enforced;

Query is running:   0%|          |

""


# I'm losing my mind a bit here because I can't find any solution to this in our github repo or even on stack exchange.  I have a table where both the team_away and team_home fields reference the same primary key in the Team_Identification table.  Bigquery will not let me do this, no matter what I try.  I'm stumped here.

In [213]:
#%%bigquery
#alter table football_dataset_int.Spreadspoke_Scores_Updated add constraint team_home_fk foreign key (team_away) references football_dataset_int.Team_Identification (team_name) not enforced;

In [214]:
%%bigquery
alter table football_dataset_int.Team_Conference add constraint team_fk foreign key (team_name) references football_dataset_int.Team_Identification (team_name) not enforced;

Query is running:   0%|          |

""


In [215]:
%%bigquery
alter table football_dataset_int.Team_Identification add constraint team_fk foreign key (team_name) references football_dataset_int.Team_Conference (team_name) not enforced;

Query is running:   0%|          |

""


# Cleaning up Temp Tables

In [216]:
%%bigquery
drop table if exists football_dataset_int.Stadiums_Updated;
drop table if exists football_dataset_int.Stadiums_enrichment;
drop table if exists football_dataset_int.stadium_mapping_temp_;
drop table if exists football_dataset_int.tmp_Stadiums;

Query is running:   0%|          |

""
